# The FSRS algorithm

this notebook contains an exploratory implementation of the FSRS algorithm. This algorithm was developed initally by Jarret Ye et.al and a description of it can be found [here](https://github.com/open-spaced-repetition/fsrs4anki/wiki/The-Algorithm). The goal here is to familiarize myself with it enough such that I can implement a rust based version in it for [minra](https://github.com/lakrestofer/minra).



Here I'm essentially doing nothing but reading the related wiki note linked above.

The algorithm models each review item as a 3 valued tuple containing its retrievability ($R$), stability ($S$) and difficulty ($D$). The retrievability is the probability that the item will be recalled at the current timepoint $t$. The stability is the number of days required to drop the retrievability from 100% to the wanted retrievability probability (usually around 90%). The difficulty is some number between 1 and 10 that represents how difficult an item is to remember (makes sense). 

### Initial item parameters

the model is built up of various formulas that use some arbitrary parameters (contained in the vector $W$ with parameters $w_i$) whose values has been set by fitting the model to real world data. This also means that we will be able to further tune them when we get access to more data.

An initial review item has its $(R,S,D)$ values set through the first review event ($G$ for "grade").

initial stability:
$$
S_0(G) = w_{G-1}
$$

initial difficulty:
$$
D_0(G) = w_4 - (G - 3) \cdot w_5
$$

and retrievability (which is not a constant but function over time t):
$$
R(t,S) = (1 + \frac{t}{9 \cdot S})^{-1}
$$


In [17]:
from typing import Callable
from dataclasses import dataclass
import jdc # allows for glorious multicell implementations of a python class!
from math import exp

In [5]:
# the parameters set through fitting the model to real world anki data.
W = [0.4, 0.6, 2.4, 5.8, 4.93, 0.94, 0.86, 0.01, 1.49, 0.14, 0.94, 2.18, 0.05, 0.34, 1.26, 0.29, 2.61]

# checks if value is valid
def val_g(g: int, f: Callable[[int], int]) -> int:
    if (g < 0 or 4 < g):
        raise ValueError("Nonvalid grade. Grade must be between 1 and 4 (inclusive range)")
    return f(g)

# initial stability
def s0(g: int) -> int:
    return val_g(g, lambda g: W[g - 1])

# initial difficulty
def d0(g: int) -> int:
    return val_g(g, lambda g: W[4] - (g - 3) * W[5])

we define an initial review item class using the above formulas

In [8]:
@dataclass
class ReviewItem:
    s: int
    d: int
    
    def init_item(g: int) -> "ReviewItem":
        s = s0(g)
        d = d0(g)
        return ReviewItem(s,d)
    


We then want a way to compute how the retrievability changes for the item over time. We recall the formula.
$$
R(t,S) = (1 + \frac{t}{9 \cdot S})^{-1}
$$
which we can then add to our review item class

In [13]:
%%add_to ReviewItem
def r(self, t: int) -> float:
    """
    returns the probability that this item will be recalled after t days
    :param int t: Number of days since last review
    """
    return 1 / (1 + (t / (9 * self.s)))


This then gives us a way to decide when we should review the item. When the retrievability reaches the target probability we review, produce a grade g that tells us how well we recalled the item and then update our (or create a new) ReviewItem.

For this we need to compute the new stability and difficulty given the review grade g.

The new difficulty is computed as below:
$$
D'(D,G) = w_7 \cdot D_0(3) + (1 - w_7) * (D - w_6 \cdot (G - 3))
$$

In [14]:
%%add_to ReviewItem
def new_d(self, g: int) -> int:
    return W[7] * d0(3) + (1 - W[7]) * (self.d - W[6] * (g - 3))

given that the review was successful the new stability is computed as:

$$
    S'_r(D,S,R,G) = S \cdot (e^{w_8} \cdot (11 - D) \cdot S^{-W_9} \cdot (e^{w_{10}\cdot(1 - R)}) \cdot w_{15}(\text{if}\, G = 2) \cdot w_{16}(\text{if}\, G = 4) + 1)
$$

In [21]:
%%add_to ReviewItem
def new_s_suc(self, g: int, t: int) -> int:
    # current recall probability
    cr = self.r(t)
    # grade factor. just some number based on the grade.
    gf = 1 if g == 3 else W[15] if g == 2 else W[16]
    # the factor with which we scale the original stability
    d = exp(W[8]) * (11 - self.d) * self.s**(-W[9]) * (exp(W[10] * (1 - cr)) - 1) * gf + 1
    return self.s * d

in the case that the review failed (we could not recall the item) with a resulting g of 1 then the stability is compute as:
$$
S'_f(D,S,R) = w_{11} \cdot D^{-w_{12}} \cdot ((S + 1)^{w_{13}} - 1) \cdot e^{w_{14} \cdot (1 - R)}
$$

In [22]:
%%add_to ReviewItem
def new_s_fail(self, t: int) -> int:
    cr = self.r(t)
    return W[11] * self.d**(-W[12]) * ((self.s + 1)**(W[13]) - 1) * exp(W[14] * (1 - cr))

which we can provide a nicer interface through

In [23]:
%%add_to ReviewItem
def new_s(self, g: int, t:int) -> int:
    return val_g(g, lambda g: new_s_fail(g) if g == 1 else new_s_suc(g))

In [25]:
%%add_to ReviewItem
def review(self, g: int, t: int) -> "ReviewItem":
    """
    computes the new stability and difficulty of an item given the result g of a review event
    """
    nd = self.new_d(g)
    ns = self.new_s(g, t)
    return ReviewItem(nd, ns)

this is enough to fully models the creation of review items and the reviews themselves!

In [ ]:
item = ReviewItem.init_item(3) # an ordinary item